# Language modelling
Mateusz Wojtulewicz

## 1. Read the documentation of [Language modelling in the Transformers](https://huggingface.co/docs/transformers/task_summary#language-modeling).



In [78]:
from IPython.display import clear_output 

!pip install transformers
!pip install sacremoses

clear_output()
print("Done.")

Done.


In [23]:
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    AutoModelWithLMHead, 
    BertTokenizer, 
    BertForMaskedLM, 
    AutoModel,
    pipeline
)

from pprint import pprint

## 2. Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository.

I'm using a Polish BERT model, a Polish GPT-2 model and a multilingual XLM-RoBERT model.

In [74]:
polbert = {
    "tokenizer": BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1"),
    "model": BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
}

def use_polbert(*parts):
    unmasker = pipeline(
        "fill-mask", 
        model=polbert["model"], 
        tokenizer=polbert["tokenizer"]
    )
    sentence = f" {unmasker.tokenizer.mask_token} ".join(parts)
    outputs = unmasker(sentence)
    for i, out in enumerate(outputs):
        print(f"{i+1} (score = {out['score']:.3f}): {out['sequence']}")

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
use_polbert("Słowacki wielkim", "był.")

1 (score = 0.112): słowacki wielkim poeta był.
2 (score = 0.095): słowacki wielkim człowiekiem był.
3 (score = 0.066): słowacki wielkim bohaterem był.
4 (score = 0.056): słowacki wielkim kutasem był.
5 (score = 0.048): słowacki wielkim nie był.


In [76]:
use_polbert("", "wrze w 100 stopniach Celsjusza.")

1 (score = 0.275): woda wrze w 100 stopniach celsjusza.
2 (score = 0.112): temperatura wrze w 100 stopniach celsjusza.
3 (score = 0.045): nie wrze w 100 stopniach celsjusza.
4 (score = 0.041): tlen wrze w 100 stopniach celsjusza.
5 (score = 0.037): - wrze w 100 stopniach celsjusza.


In [84]:
papuga = {
    "model": AutoModelWithLMHead.from_pretrained("flax-community/papuGaPT2"),
    "tokenizer": AutoTokenizer.from_pretrained("flax-community/papuGaPT2")
}

def use_papuga(sentence: str):
    input_ids = papuga["tokenizer"].encode(sentence, return_tensors="pt")

    outputs = papuga["model"].generate(
        input_ids,
        do_sample=True, 
        max_new_tokens=15, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=5
    )
    sentences = [
        papuga["tokenizer"].decode(out, skip_special_tokens=True) 
        for out in outputs
    ]

    clear_output()
    for i, s in enumerate(sentences):
        print(f"{i+1}: {s}")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [85]:
use_papuga("W 100 stopniach Celsjusza wrze")

1: W 100 stopniach Celsjusza wrze na rozgrzanej grzałce, co powoduje wzrost temperatury o 20 stopni Celsjusza.
2: W 100 stopniach Celsjusza wrze z góry i w środku. Jak ostygnie - jest to tzw. wrzą
3: W 100 stopniach Celsjusza wrze. Następnie do ciasta dodajemy startą czekoladę i mieszamy, aż do połączenia składników
4: W 100 stopniach Celsjusza wrzeć należy 1 minutę i 10 sekund. Dodać łyżkę mąki. Mieszamy mąkę
5: W 100 stopniach Celsjusza wrzecionko, by je podgrzać. Podgrzewam wodę do temperatury


In [86]:
use_papuga("Bitwa pod Grunwaldem miała miejsce w")

1: Bitwa pod Grunwaldem miała miejsce w dniach 11 – 15 lipca 1657 r. Na polach Grunwaldu miały miejsce
2: Bitwa pod Grunwaldem miała miejsce w roku 1210. Do dziś nie wiadomo, kto pokonał Grunwald – czy
3: Bitwa pod Grunwaldem miała miejsce w 1094 roku i trwała 12 dni, podczas których armia polsko-litewska
4: Bitwa pod Grunwaldem miała miejsce w latach 1410–1411. Z czasem była jednak bardziej spektakularna i
5: Bitwa pod Grunwaldem miała miejsce w 1656r. Pod Grunwaldem w 1657r. wojska krzyżackie


In [81]:
roberta = {
    "model": AutoModelForMaskedLM.from_pretrained("xlm-roberta-large"),
    "tokenizer": AutoTokenizer.from_pretrained("xlm-roberta-large")
}

def use_roberta(*parts):
    unmasker = pipeline(
        "fill-mask", 
        model=roberta["model"], 
        tokenizer=roberta["tokenizer"]
    )
    sentence = f" {unmasker.tokenizer.mask_token} ".join(parts)
    outputs = unmasker(sentence)
    for i, out in enumerate(outputs):
        print(f"{i+1} (score = {out['score']:.3f}): {out['sequence']}")

In [82]:
use_roberta("Na górze", ", na dole fiołki, a my się kochamy jak dwa aniołki.")

1 (score = 0.154): Na górze słońce, na dole fiołki, a my się kochamy jak dwa aniołki.
2 (score = 0.150): Na górze śnieg, na dole fiołki, a my się kochamy jak dwa aniołki.
3 (score = 0.139): Na górze kwiaty, na dole fiołki, a my się kochamy jak dwa aniołki.
4 (score = 0.080): Na górze góry, na dole fiołki, a my się kochamy jak dwa aniołki.
5 (score = 0.018): Na górze róż, na dole fiołki, a my się kochamy jak dwa aniołki.


In [83]:
use_polbert("Na górze", ", na dole fiołki, a my się kochamy jak dwa aniołki.")

1 (score = 0.236): na gorze roze, na dole fiołki, a my sie kochamy jak dwa aniołki.
2 (score = 0.052): na gorze kwiaty, na dole fiołki, a my sie kochamy jak dwa aniołki.
3 (score = 0.014): na gorze lody, na dole fiołki, a my sie kochamy jak dwa aniołki.
4 (score = 0.013): na gorze czekoladki, na dole fiołki, a my sie kochamy jak dwa aniołki.
5 (score = 0.012): na gorze jabłka, na dole fiołki, a my sie kochamy jak dwa aniołki.


## 3. Devise a method to test if the langage model understands Polish cases. Create sentences for each case.

### 1. Mianownik

In [87]:
use_polbert("Słoń to największe", "na świecie.")

1 (score = 0.473): słon to najwieksze zwierze na swiecie.
2 (score = 0.102): słon to najwieksze stworzenie na swiecie.
3 (score = 0.049): słon to najwieksze zło na swiecie.
4 (score = 0.048): słon to najwieksze jabłko na swiecie.
5 (score = 0.032): słon to najwieksze miasto na swiecie.


In [88]:
use_roberta("Słoń to największe", "na świecie.")

1 (score = 0.614): Słoń to największe słońce na świecie.
2 (score = 0.118): Słoń to największe źródło na świecie.
3 (score = 0.059): Słoń to największe światło na świecie.
4 (score = 0.053): Słoń to największe ciało na świecie.
5 (score = 0.033): Słoń to największe planeta na świecie.


In [89]:
use_papuga("Największe zwierze na świecie to")

1: Największe zwierze na świecie to pies rasy husky, który żyje tylko przez około 40 sekund. W tym
2: Największe zwierze na świecie to świnka morska. Jest to zwierzę z gatunku żbików. Można spotkać
3: Największe zwierze na świecie to lwy i delfiny. Do ich największych wrogów należą lwy morskie,
4: Największe zwierze na świecie to pies, który w ten sposób został udomowiony, jest bardzo pło
5: Największe zwierze na świecie to ssaki kopytne. Są one tak zróżnicowane, że stanowią o ich charakterze


### 2. Dopełniacz

In [91]:
use_polbert("Od tygodnia szukam", ", nie wiem gdzie się podział.")

1 (score = 0.045): od tygodnia szukam syna, nie wiem gdzie sie podział.
2 (score = 0.040): od tygodnia szukam chłopaka, nie wiem gdzie sie podział.
3 (score = 0.038): od tygodnia szukam przyjaciela, nie wiem gdzie sie podział.
4 (score = 0.028): od tygodnia szukam brata, nie wiem gdzie sie podział.
5 (score = 0.025): od tygodnia szukam ciała, nie wiem gdzie sie podział.


In [92]:
use_roberta("Od tygodnia szukam", ", nie wiem gdzie się podział.")

1 (score = 0.085): Od tygodnia szukam go, nie wiem gdzie się podział.
2 (score = 0.072): Od tygodnia szukam kompa, nie wiem gdzie się podział.
3 (score = 0.063): Od tygodnia szukam telefonu, nie wiem gdzie się podział.
4 (score = 0.039): Od tygodnia szukam jej, nie wiem gdzie się podział.
5 (score = 0.029): Od tygodnia szukam syna, nie wiem gdzie się podział.


In [93]:
use_papuga("Nie mogę znaleźć swojego")

1: Nie mogę znaleźć swojego. A może on jest już schowany do kieszeni i nie ma go w
2: Nie mogę znaleźć swojego laptopa. Nie wiem, czy go oddam. Nie mam też komputera stacjonarnego
3: Nie mogę znaleźć swojego imienia, które było by dla ciebie ciekawe i które cię fascynowało, a
4: Nie mogę znaleźć swojego stylu pisania i stylu nie mogę. Moje dialogi są nie do końca spójne
5: Nie mogę znaleźć swojego zdjęcia? To jest mój ulubiony blog. I mój ulubiony blog o modzie,


### 3. Celownik

In [95]:
use_polbert("Dałem swojej", "piłkę.")

1 (score = 0.446): dałem swojej mamie piłke.
2 (score = 0.179): dałem swojej matce piłke.
3 (score = 0.162): dałem swojej dziewczynie piłke.
4 (score = 0.044): dałem swojej siostrze piłke.
5 (score = 0.020): dałem swojej dziewczynce piłke.


In [97]:
use_roberta("Dałem swojej", "piłkę.")

1 (score = 0.250): Dałem swojej stronie piłkę.
2 (score = 0.111): Dałem swojej szkole piłkę.
3 (score = 0.040): Dałem swojej młodzieży piłkę.
4 (score = 0.039): Dałem swojej firmie piłkę.
5 (score = 0.032): Dałem swojej grze piłkę.


In [98]:
use_papuga("Przyglądam się")

1: Przyglądam się jego pracy w internecie i muszę przyznać, że mnie też ona zachwyciła.
2: Przyglądam się im od dłuższego czasu...Wg mnie to jest fajny zawód. Nie ma
3: Przyglądam się sobie cotygodniowym felietonom. Tak, bardzo często widzę w nich
4: Przyglądam się w ostatnim momencie (co za szczęście). W końcu za chwilę się obudzę
5: Przyglądam się temu człowiekowi w nadziei, że i on sam będzie miał dzieci i rodzinę jak


### 4. Biernik

In [105]:
use_polbert("Moja mam kupiła mi", ".")

1 (score = 0.091): moja mam kupiła mi go.
2 (score = 0.089): moja mam kupiła mi to.
3 (score = 0.044): moja mam kupiła mi je.
4 (score = 0.030): moja mam kupiła mi samochod.
5 (score = 0.024): moja mam kupiła mi loda.


In [106]:
use_roberta("Moja mama kupiła mi", ".")

1 (score = 0.065): Moja mama kupiła mi samochód.
2 (score = 0.059): Moja mama kupiła mi książkę.
3 (score = 0.036): Moja mama kupiła mi to.
4 (score = 0.035): Moja mama kupiła mi dom.
5 (score = 0.032): Moja mama kupiła mi rower.


In [107]:
use_papuga("Moja mama kupiła mi")

1: Moja mama kupiła mi w tym sklepie i polecam z całego serca. A jak mam jeszcze problem z
2: Moja mama kupiła mi taki model, więc od razu narobiłam batonów na śniadanie. Są
3: Moja mama kupiła mi te świece za grosze i są one dostępne praktycznie u każdej z Was. Uważam
4: Moja mama kupiła mi dzisiaj nowy olej i w sumie nie wiem czy bym z niego wyrobiła.
5: Moja mama kupiła mi na urodziny nowy zestaw do manicure hybrydowego – krem o nazwie Serene


### 5. Narzędnik

In [108]:
use_polbert("Ta dziewczyna z długimi", "była bardzo ładna.")

1 (score = 0.918): ta dziewczyna z długimi włosami była bardzo ładna.
2 (score = 0.062): ta dziewczyna z długimi nogami była bardzo ładna.
3 (score = 0.007): ta dziewczyna z długimi oczami była bardzo ładna.
4 (score = 0.003): ta dziewczyna z długimi ustami była bardzo ładna.
5 (score = 0.001): ta dziewczyna z długimi stopami była bardzo ładna.


In [109]:
use_roberta("Ta dziewczyna z długimi", "była bardzo ładna.")

1 (score = 0.863): Ta dziewczyna z długimi włosy była bardzo ładna.
2 (score = 0.067): Ta dziewczyna z długimi włosów była bardzo ładna.
3 (score = 0.012): Ta dziewczyna z długimi włos była bardzo ładna.
4 (score = 0.011): Ta dziewczyna z długimi czasami była bardzo ładna.
5 (score = 0.008): Ta dziewczyna z długimi bikini była bardzo ładna.


In [110]:
use_papuga("Bardzo spodobała mi się ta dziewczyna z długimi")

1: Bardzo spodobała mi się ta dziewczyna z długimi włosami, która jak tylko zobaczyła jej zdjęcie powiedziała, że to już koniec jej
2: Bardzo spodobała mi się ta dziewczyna z długimi włosami! :D Nie wiem czy to dlatego że mam za długie włosy,
3: Bardzo spodobała mi się ta dziewczyna z długimi włosami... jak to się ma do włosów? Wyglądasz w niej całkiem fajnie
4: Bardzo spodobała mi się ta dziewczyna z długimi włosami i jest to jedna z najlepszych stylizacji jakie widziałam. Uwielbiam ten styl i
5: Bardzo spodobała mi się ta dziewczyna z długimi włosami, w ogóle to jej się podoba, poza tym, że miała krótkie


### 6. Miejscownik

In [111]:
use_polbert("Nie mogę przestać myśleć o", ".")

1 (score = 0.286): nie moge przestac myslec o tobie.
2 (score = 0.068): nie moge przestac myslec o niej.
3 (score = 0.055): nie moge przestac myslec o tym.
4 (score = 0.037): nie moge przestac myslec o nim.
5 (score = 0.030): nie moge przestac myslec o sobie.


In [112]:
use_roberta("Nie mogę przestać myśleć o", ".")

1 (score = 0.394): Nie mogę przestać myśleć o nim.
2 (score = 0.174): Nie mogę przestać myśleć o tym.
3 (score = 0.165): Nie mogę przestać myśleć o niej.
4 (score = 0.053): Nie mogę przestać myśleć o Nim.
5 (score = 0.022): Nie mogę przestać myśleć o nich.


In [113]:
use_papuga("Nie mogę przestać myśleć o")

1: Nie mogę przestać myśleć o tym jak wspaniale bawi się w tym miejscu. To tak, jakby się tam
2: Nie mogę przestać myśleć o tym, jak bardzo nie doceniasz mojego wkładu w edukację i dobrostanie,
3: Nie mogę przestać myśleć o tym jak wyglądała moja przygoda z tańcem i jakie moje marzenie się spełniło
4: Nie mogę przestać myśleć o sobie jak o zwierzęciu, a czuję się zupełnie jak ona. Zdą
5: Nie mogę przestać myśleć o Twoich wspaniałych zdjęciach i o Twojej twórczości :) Pozdrawiam.
Pięknie! Ale


### 7. Wołacz

In [128]:
use_polbert("O ty", "!")

1 (score = 0.132): o ty sukinsynu!
2 (score = 0.068): o ty suko!
3 (score = 0.045): o ty dupku!
4 (score = 0.042): o ty draniu!
5 (score = 0.032): o ty boze!


In [127]:
use_roberta("Święty", ", módl się za nami.")

1 (score = 0.268): Święty Boże, módl się za nami.
2 (score = 0.153): Święty Józef, módl się za nami.
3 (score = 0.061): Święty Mikołaj, módl się za nami.
4 (score = 0.043): Święty Andrzej, módl się za nami.
5 (score = 0.037): Święty Stanisław, módl się za nami.


In [131]:
use_papuga("Nie rozumiesz mnie, mój")

1: Nie rozumiesz mnie, mój chłopcze, to i ja cię proszę bardzo” - wołał Jezus na
2: Nie rozumiesz mnie, mój przyjacielu... - w jego głosie nie było nutki zadowolenia, bo jak
3: Nie rozumiesz mnie, mój mały ptaszku. Ja nie chcę, by wszyscy się dowiedzieli. Ja chcę
4: Nie rozumiesz mnie, mój Boże, tak daleki jesteś od zrozumienia, że nie mogę uwierzyć. To
5: Nie rozumiesz mnie, mój kolega to był prawdziwy leszcz. Co ciekawe miał wtedy ponad 13lat i


## 4. Devise a method to test long-range relationships such as gender. Define at least 3 sentences.

In [133]:
use_polbert("", ", którą dostałam od Kuby, to najładniejsza jaką widziałem w życiu.")

1 (score = 0.352): ta, ktora dostałam od kuby, to najładniejsza jaka widziałem w zyciu.
2 (score = 0.067): sukienka, ktora dostałam od kuby, to najładniejsza jaka widziałem w zyciu.
3 (score = 0.038): bransoletka, ktora dostałam od kuby, to najładniejsza jaka widziałem w zyciu.
4 (score = 0.037): dziewczyna, ktora dostałam od kuby, to najładniejsza jaka widziałem w zyciu.
5 (score = 0.031): kurtka, ktora dostałam od kuby, to najładniejsza jaka widziałem w zyciu.


In [134]:
use_roberta("", ", którą dostałam od Kuby, to najładniejsza jaką widziałem w życiu.")

1 (score = 0.388): Ta, którą dostałam od Kuby, to najładniejsza jaką widziałem w życiu.
2 (score = 0.082): Karta, którą dostałam od Kuby, to najładniejsza jaką widziałem w życiu.
3 (score = 0.051): Maska, którą dostałam od Kuby, to najładniejsza jaką widziałem w życiu.
4 (score = 0.027): ta, którą dostałam od Kuby, to najładniejsza jaką widziałem w życiu.
5 (score = 0.021): Kawa, którą dostałam od Kuby, to najładniejsza jaką widziałem w życiu.


In [135]:
use_papuga("Nigdy nie widziałam ładniejszej")

1: Nigdy nie widziałam ładniejszej i co najważniejsze - na luzie, bo po raz pierwszy nie byłam w
2: Nigdy nie widziałam ładniejszej kobiety. Jestem z Polski. Czy to na plaży? Tak. Z chęcią
3: Nigdy nie widziałam ładniejszej i jeszcze lepiej zachowanej części ulicy z czasów okupacji. A potem się z
4: Nigdy nie widziałam ładniejszej kobiety. Za każdym razem, kiedy ją widzę, mam ochotę uciec z tą
5: Nigdy nie widziałam ładniejszej! 🙂 Ja też od wielu lat szyję głównie na potrzeby własne, więc nawet


In [137]:
use_polbert("Wszystkie", "miały białe ubrania i czarne buty.")

1 (score = 0.688): wszystkie ofiary miały białe ubrania i czarne buty.
2 (score = 0.128): wszystkie kobiety miały białe ubrania i czarne buty.
3 (score = 0.048): wszystkie dziewczyny miały białe ubrania i czarne buty.
4 (score = 0.025): wszystkie dzieci miały białe ubrania i czarne buty.
5 (score = 0.009): wszystkie one miały białe ubrania i czarne buty.


In [138]:
use_roberta("Wszystkie", "miały białe ubrania i czarne buty.")

1 (score = 0.647): Wszystkie kobiety miały białe ubrania i czarne buty.
2 (score = 0.139): Wszystkie dzieci miały białe ubrania i czarne buty.
3 (score = 0.027): Wszystkie modele miały białe ubrania i czarne buty.
4 (score = 0.010): Wszystkie trzy miały białe ubrania i czarne buty.
5 (score = 0.009): Wszystkie osoby miały białe ubrania i czarne buty.


In [139]:
use_papuga("W zamkniętym przedziale znajdują zasztyletowanego")

1: W zamkniętym przedziale znajdują zasztyletowanego w nocy na ulicy. Jest kilka osób, które go szukają, nikt nie
2: W zamkniętym przedziale znajdują zasztyletowanego, po długim i męczącym tygodniu. W ciągu ostatnich dni na korytarzach
3: W zamkniętym przedziale znajdują zasztyletowanego. Na drzwiach od kabiny znajduje się krata z otworami drzwiowymi,
4: W zamkniętym przedziale znajdują zasztyletowanego młodzieńca, jego matkę oraz starszego brata z czwórką małych dzieci, którzy
5: W zamkniętym przedziale znajdują zasztyletowanego psa, który uciekł z ulicy, z pomocą której uratowano go przed śmiercią


In [166]:
use_polbert("Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie", ".")

1 (score = 0.237): gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym sie nie dowiedział.
2 (score = 0.084): gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym sie nie martwił.
3 (score = 0.075): gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym sie nie przyznał.
4 (score = 0.075): gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym sie nie zgodził.
5 (score = 0.037): gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym sie nie zdziwił.


In [161]:
use_roberta("Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie", ".")

1 (score = 0.415): Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zmienił.
2 (score = 0.076): Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zdecydował.
3 (score = 0.068): Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie wybrał.
4 (score = 0.057): Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie pojawił.
5 (score = 0.046): Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie przygotował.


In [164]:
use_papuga("Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie")

1: Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zgodził. Tak jak ja jestem przekonany, że jest to część procesu myślenia i
2: Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie martwił, że przez brak wiary straciłem swój udział w budowaniu tej ziemi
3: Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zastanawiał.
Co do tego, że coś jest dobre, to ja nie
4: Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zgodził! (1) A wtedy Pan odpowiedział mi: Zaprawdę, mówię
5: Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zgodził, bo gdybym wiedział z łatwością to, co wiem dziś, byłbym głup


## 5. Check if the model captures real-world knowledge. Define at lest 3 sentences.

In [146]:
sentence = ["Bitwa pod Grunwaldem miała miejsce w", "roku."]

In [147]:
use_polbert(*sentence)

1 (score = 0.060): bitwa pod grunwaldem miała miejsce w 1920 roku.
2 (score = 0.037): bitwa pod grunwaldem miała miejsce w 1919 roku.
3 (score = 0.037): bitwa pod grunwaldem miała miejsce w 1939 roku.
4 (score = 0.029): bitwa pod grunwaldem miała miejsce w 1792 roku.
5 (score = 0.029): bitwa pod grunwaldem miała miejsce w 1945 roku.


In [149]:
use_roberta(*sentence)

1 (score = 0.142): Bitwa pod Grunwaldem miała miejsce w 1918 roku.
2 (score = 0.138): Bitwa pod Grunwaldem miała miejsce w 1945 roku.
3 (score = 0.127): Bitwa pod Grunwaldem miała miejsce w 1939 roku.
4 (score = 0.081): Bitwa pod Grunwaldem miała miejsce w 1914 roku.
5 (score = 0.056): Bitwa pod Grunwaldem miała miejsce w 1920 roku.


In [151]:
use_papuga(sentence[0])

1: Bitwa pod Grunwaldem miała miejsce w 1421 roku, a jej pomysłodawcą był Jan Długosz. Jednak dopiero
2: Bitwa pod Grunwaldem miała miejsce w latach 1410-1414. W tym czasie państwo krzyżackie było już
3: Bitwa pod Grunwaldem miała miejsce w roku 1412, kiedy to w bitwie pod Grunwaldem wojska krzyżackie straciły
4: Bitwa pod Grunwaldem miała miejsce w roku 1410. Król Władysław Jagiełło ogłosił ją w 1421 roku.
5: Bitwa pod Grunwaldem miała miejsce w roku 1454, a więc mniej więcej w tym samym czasie, w którym


In [152]:
sentence = ["Największym kontynentem jest", "."]

In [153]:
use_polbert(*sentence)

1 (score = 0.118): najwiekszym kontynentem jest australia.
2 (score = 0.049): najwiekszym kontynentem jest afryka.
3 (score = 0.040): najwiekszym kontynentem jest ameryka.
4 (score = 0.030): najwiekszym kontynentem jest anglia.
5 (score = 0.030): najwiekszym kontynentem jest brazylia.


In [154]:
use_roberta(*sentence)

1 (score = 0.250): Największym kontynentem jest Australia.
2 (score = 0.132): Największym kontynentem jest Madagaskar.
3 (score = 0.118): Największym kontynentem jest Europa.
4 (score = 0.044): Największym kontynentem jest USA.
5 (score = 0.034): Największym kontynentem jest India.


In [155]:
use_papuga(sentence[0])

1: Największym kontynentem jest oczywiście Australia, a konkretnie jej północna część - Australia Południowa (południ
2: Największym kontynentem jest Australia, a w szczególności okolice Melbourne. Tamtejsza natura jest
3: Największym kontynentem jest Afryka Środkowa na wysokości 1200-1300 m n.p.m
4: Największym kontynentem jest Azja – tam ma swój początek Azję, w której można zaobserwować największą ilość
5: Największym kontynentem jest właśnie Afryka.
W tym regionie, jeśli się nie chce, można znaleźć


In [156]:
sentence = ["Jedną z planet Układu Słonecznego jest", "."]

In [157]:
use_polbert(*sentence)

1 (score = 0.137): jedna z planet układu słonecznego jest ziemia.
2 (score = 0.048): jedna z planet układu słonecznego jest andromeda.
3 (score = 0.041): jedna z planet układu słonecznego jest planeta.
4 (score = 0.040): jedna z planet układu słonecznego jest wenus.
5 (score = 0.028): jedna z planet układu słonecznego jest tutaj.


In [158]:
use_roberta(*sentence)

1 (score = 0.362): Jedną z planet Układu Słonecznego jest Saturn.
2 (score = 0.309): Jedną z planet Układu Słonecznego jest Mars.
3 (score = 0.085): Jedną z planet Układu Słonecznego jest Jupiter.
4 (score = 0.078): Jedną z planet Układu Słonecznego jest Neptun.
5 (score = 0.027): Jedną z planet Układu Słonecznego jest Luna.


In [159]:
use_papuga(sentence[0])

1: Jedną z planet Układu Słonecznego jest Wenus, zwana także w skrócie Wenus, położona na granicy między Zie
2: Jedną z planet Układu Słonecznego jest Księżyc, który jest obecnie najbardziej oddalonym od Ziemi ciałem niebieskim, a w
3: Jedną z planet Układu Słonecznego jest Wenus, która jest uważana za najbardziej znaną planetę Układu Słonecznego
4: Jedną z planet Układu Słonecznego jest kometa (Gaia), a dokładniej ich najbliższa rodzina komet.
5: Jedną z planet Układu Słonecznego jest planeta – Słońce. Jej nazwa nawiązuje do nazwy planety oraz jej koloru.


## 7. Answer the following questions:

### a. Which of the models produced the best results?

In my opinion the best results were produces by Polish BERT. Most of its guesses were matching the context and grammar.

### b. Was any of the models able to capture Polish grammar?

Papuga was generating sentences with mostly correct grammar. Polbert was excellent in polish cases, while Robert did some mistakes.

### c. Was any of the models able to capture long-distant relationships between the words?

Every model did a good job in capturing long-distance relationships beetwen the words (such as gender).

### d. Was any of the models able to capture world knowledge?

None of the models has a very accurate world knowledge, such as what is the biggest continent (although most guesses were continents) or in which year did the Grunwald battle take place (apart from Papuga in some guesses). But when it came to naming planets, the guesses where much closer.

### e. What are the most striking errors made by the models?

The most striking errors are when the model did not understand the context, and, although the form of the word is correct, it doesn't fit in the sentence. For example `Słoń to największe źródło na świecie.`, `Ta dziewczyna z długimi czasami była bardzo ładna.`. Most of such mistakes were made by the RoBERT model, which is the biggest of all three. This might be because it was trained on 94 languages while the remaining two only on Polish language.